In [0]:
import tensorflow as tf
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import layers
from tensorflow.keras import Model
from model_files import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import sys

seed = 42

In [0]:
def load_dataset(datasetPath):
    # grab the paths to all images in our dataset directory, then
    # initialize our lists of images
    imagePaths = list(paths.list_images(datasetPath))
    data = []

    # loop over the image paths
    for imagePath in imagePaths:

        # load the image and resize it to be a fixed 128x128 pixels
        # ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (128, 128))

        # add the image to the data lists
        data.append(image)

    # return the data list as a NumPy array
    return np.array(data, dtype= "float32")

In [13]:
print("[INFO] loading data...")
fireData = load_dataset(config.FIRE_PATH)
nonFireData = load_dataset(config.NON_FIRE_PATH)

[INFO] loading data...


In [0]:
# construct the class labels for the data
fireLabels = np.ones((fireData.shape[0],))
nonFireLabels = np.zeros((nonFireData.shape[0],))

In [0]:
# stack the fire data with the non-fire data, then scale the data
# to the range [0, 1]
data = np.vstack([fireData, nonFireData])
labels = np.hstack([fireLabels, nonFireLabels])

data /= 255

In [0]:
# perform one-hot encoding on the labels and account for skew in the
# labeled data
labels = to_categorical(labels, num_classes=2)
classTotals = labels.sum(axis=0)
classWeight = classTotals.max() / classTotals

In [0]:
# construct the training and testing split
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=config.TEST_SPLIT, random_state=seed)

# initialize the training data augmentation object
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import keras 

# Parameter values combinations for hypertuning of parameters

epochs = [30, 50, 70]
dropout = [0.1, 0.2, 0.3]
learning_rates = [0.001, 0.01, 0.1]
pre_trained_model = InceptionV3(input_shape = (128,128,3),
                                include_top = False,
                                weights = 'imagenet'
                                )

for layer in pre_trained_model.layers:
  layer.trainable = False

optimum_params = {}

for epoch in epochs:
  for drop in dropout:
    for l_rate in learning_rates:
      
      params = 'Learning Rate: ' + str(l_rate) + ' | ' + 'Dropout: ' + str(drop) + ' | ' 'Epochs: '+ str(epoch)
      print(params)
      opt = SGD(lr = l_rate, 
                momentum=0.9,
                decay= l_rate / epoch)
      
      # Flatten the output layer to 1 Dimension
      x = layers.Flatten()(pre_trained_model.output)
      # Add a Fully Connected Layer with 1024 hidden units/neurons and ReLU activation function
      x = layers.Dense(1024, activation = 'relu')(x)
      # Add a Dropout layer with rate of 0.2 
      x = layers.Dropout(drop)(x)
      # Add a final Dense Layer with sigmoid activation function for classification
      x = layers.Dense(2, activation = 'sigmoid')(x)


      model = Model(pre_trained_model.input, x)
      model.compile(optimizer= opt,
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])


      # train the network
      print("[INFO] training network...")
      H = model.fit_generator(aug.flow(trainX, trainY, batch_size=config.BATCH_SIZE),
                              validation_data= (testX, testY),
                              steps_per_epoch=trainX.shape[0] // config.BATCH_SIZE,
                              epochs= epoch,
                              verbose=1)
      
      accuracy = H.history["val_accuracy"][epoch-1]
      optimum_params[params] = accuracy

Learning Rate: 0.001 | Dropout: 0.1 | Epochs: 30
[INFO] training network...
Epoch 1/30
46/46 [==============================] - 15s 319ms/step - loss: 0.5173 - accuracy: 0.7843 - val_loss: 0.3010 - val_accuracy: 0.8881
Epoch 2/30
46/46 [==============================] - 12s 261ms/step - loss: 0.3446 - accuracy: 0.8652 - val_loss: 0.2908 - val_accuracy: 0.8861
Epoch 3/30
46/46 [==============================] - 12s 262ms/step - loss: 0.3132 - accuracy: 0.8785 - val_loss: 0.2563 - val_accuracy: 0.8991
Epoch 4/30
46/46 [==============================] - 12s 258ms/step - loss: 0.2852 - accuracy: 0.8890 - val_loss: 0.3020 - val_accuracy: 0.8861
Epoch 5/30
46/46 [==============================] - 12s 261ms/step - loss: 0.3029 - accuracy: 0.8826 - val_loss: 0.2546 - val_accuracy: 0.9021
Epoch 6/30
46/46 [==============================] - 12s 263ms/step - loss: 0.2697 - accuracy: 0.8945 - val_loss: 0.2348 - val_accuracy: 0.9041
Epoch 7/30
46/46 [==============================] - 12s 263ms/step

In [0]:
print("Validation Accuracies for different combinations of hyperparameters: ", optimum_params.values())

Validation Accuracies for different combinations of hyperparameters:  dict_values([0.9210789203643799, 0.9230769276618958, 0.901098906993866, 0.9030969142913818, 0.919080913066864, 0.8731268644332886, 0.9130868911743164, 0.9170829057693481, 0.8961039185523987, 0.9270729422569275, 0.9270729422569275, 0.8901098966598511, 0.9140859246253967, 0.901098906993866, 0.9020978808403015, 0.9180819392204285, 0.9070929288864136, 0.8991008996963501, 0.9270729422569275, 0.9270729422569275, 0.9030969142913818, 0.928071916103363, 0.9180819392204285, 0.9180819392204285, 0.9160839319229126, 0.9220778942108154, 0.9040958881378174])


In [0]:
import operator
print("Optimum parameter Values: ", max(optimum_params.items(), key=operator.itemgetter(1))[0])

Optimum parameter Values:  Learning Rate: 0.001 | Dropout: 0.2 | Epochs: 70


### Lets rerun the training for the optimal parameter values

In [18]:
epoch = 70
drop = 0.2
l_rate = 0.001

from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape = (128,128,3),
                                include_top = False,
                                weights = 'imagenet'
                                )

for layer in pre_trained_model.layers:
  layer.trainable = False


opt = SGD(lr = l_rate, 
                momentum=0.9,
                decay= l_rate / epoch)
      
      # Flatten the output layer to 1 Dimension
x = layers.Flatten()(pre_trained_model.output)
      # Add a Fully Connected Layer with 1024 hidden units/neurons and ReLU activation function
x = layers.Dense(1024, activation = 'relu')(x)
      # Add a Dropout layer with rate of 0.2 
x = layers.Dropout(drop)(x)
      # Add a final Dense Layer with sigmoid activation function for classification
x = layers.Dense(2, activation = 'sigmoid')(x)


model = Model(pre_trained_model.input, x)
model.compile(optimizer= opt,
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])


      # train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=config.BATCH_SIZE),
                              validation_data= (testX, testY),
                              steps_per_epoch=trainX.shape[0] // config.BATCH_SIZE,
                              epochs= epoch,
                              verbose=1)

87916544/87910968 [==============================] - 2s 0us/step
[INFO] training network...
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/70
46/46 [==============================] - 14s 312ms/step - loss: 0.5410 - accuracy: 0.7914 - val_loss: 0.3240 - val_accuracy: 0.8821
Epoch 2/70
46/46 [==============================] - 12s 251ms/step - loss: 0.3656 - accuracy: 0.8635 - val_loss: 0.3006 - val_accuracy: 0.8851
Epoch 3/70
46/46 [==============================] - 12s 256ms/step - loss: 0.3693 - accuracy: 0.8591 - val_loss: 0.2663 - val_accuracy: 0.8961
Epoch 4/70
46/46 [==============================] - 12s 254ms/step - loss: 0.2996 - accuracy: 0.8839 - val_loss: 0.2505 - val_accuracy: 0.9061
Epoch 5/70
46/46 [==============================] - 12s 257ms/step - loss: 0.2849 - accuracy: 0.8901 - val_loss: 0.2601 - val_accuracy: 0.8981
Epoch 6/70
46/46 [==============================] - 12s 254ms/step - loss: 0.2885 - accuracy: 0.8911 - val_loss: 0.25

In [0]:
## evaluate the network and show a classification report
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=config.BATCH_SIZE)

print(classification_report(testY.argmax(axis=1),
      predictions.argmax(axis=1), target_names=config.CLASSES))

[INFO] evaluating network...
              precision    recall  f1-score   support

    Non-Fire       0.94      0.96      0.95       647
        Fire       0.93      0.88      0.91       354

    accuracy                           0.94      1001
   macro avg       0.93      0.92      0.93      1001
weighted avg       0.93      0.94      0.93      1001



In [0]:
# construct a plot that plots and saves the training history
N = np.arange(0, config.NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc = "lower left")
plt.show()